# Reply generation

In [1]:
from llama_cpp import Llama
import os
import urllib.request
import json

## I/O processing functions

In [2]:
# Email.json input processing
def input_email_processing(file_name):
    # Using 'with' statement to automatically close the file when done
    with open(file_name) as file:
        # Load JSON object from the file
        data = json.load(file)

        # Combine subject and body into acceptable format
        email = data["subject"] + " [SEP] " + data["body"]

        sender = data["sender"]

        reply_subject = "RE: " + data["subject"]

    # The file is automatically closed when the 'with' block exits

    return sender, reply_subject, email

In [3]:
# Reply.json output processing 
def output_reply_processing(sender, reply_subject, reply):
    # Create json file structure
    json_dict =  {
        "sender": sender,
        "subject": reply_subject,
        "body": reply
    }
    # Write structure to json file 
    with open("reply.json", "w") as outfile:
        json.dump(json_dict, outfile)


## LLM

In [4]:
def download_llm(url, filename):
    # Checks if the file already exists before downloading
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
        print("File downloaded successfully.")
    else:
        print("File already exists.")

# Dowloading GGML model from HuggingFace
ggml_model_url = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf"
llm_filename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

download_llm(ggml_model_url, llm_filename)

File already exists.


In [5]:
# Create model instance
llm = Llama(model_path="mistral-7b-instruct-v0.2.Q5_K_M.gguf", n_ctx=256, n_batch=126)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 l

llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0e-05
llm_load_print_meta: f_clamp_kqv      = 0.0e+00
llm_load_print_meta: f_max_alibi_bias = 0.0e+00
llm_load_print_meta: f_logit_scale    = 0.0e+00
llm_load_print_meta: n_ff             = 14336
llm_load_print_meta: n_expert         = 0
llm_load_print_meta: n_expert_used    = 0
llm_load_print_meta: causal attn      = 1
llm_load_print_meta: pooling type     = 0
llm_load_print_meta: rope type        = 0
llm_load_print_meta: rope scaling     = linear
llm_load_print_meta: freq_base_train  = 1000000.0
llm_load_print_meta: freq_scale_train = 1
llm_load_print_meta: n_yarn_orig_ctx  = 32768
llm_load_print_meta: rope_finetuned   = unknown
llm_load_print_meta: ssm_d_conv       = 0
llm_load_print_meta: ssm_d_inner      = 0
llm_load_print_meta: ssm_d_state      = 0
llm_load_print_meta: ssm_dt_rank      = 0
llm_load_prin

In [6]:
def generate_reply(
        prompt="Ask me some personal questions",  # Default prompt to start generation
        max_tokens=512,                     # Maximum number of tokens to generate
        temperature=0.4,                    # Parameter controlling the randomness of the generation
        top_p=0.5,                          # Parameter for nucleus sampling
        echo=False,                         # Whether to print intermediate steps during generation
        stop=["#"],                         # List of tokens that, if generated, will stop the generation process
):
    # Call the language model with specified parameters
    output = llm(prompt,
                 max_tokens=max_tokens,
                 temperature=temperature,
                 top_p=top_p,
                 echo=echo,
                 stop=stop,
                 )
    
    # Extract the generated text from the output
    output_text = output["choices"][0]["text"].strip()
    
    # Return the generated text
    return output_text


In [7]:
def generate_prompt_from_template(email):
    chat_prompt_template =f"""user
You are receiving a scam email and want to reply to waste the scammer's time. Craft a response to the email below:
{email}
system
Feel free to engage, ask irrelevant questions, act confused, or provide false information to waste their time. Provide only the body of the message. Keep it short. Do not use Dear Scammer as greating."""    
    return chat_prompt_template